# Averaged Perceptron

This is for a homework assignment for Mans Hulden's Machine Learning and Linguistics Course Fall 2017. It is part of the CLASIC program (computational linguistics) at CU Boulder, an interdisciplinary program housed in the Computer Science Dept and the Linguistics Dept.

Introduction:<br>
The perceptron is an artificial neuron capable of linear classification. The data are restaurant reviews from three types of restaurants, Mexican, Italian, and Chinese. Our task is to classify which type of restaurant each review comes from. The features have already been extracted beforehand. This notebook includes the training and testing of this perceptron restaurant classifier.<br>
Many of these operations could be done with packages like Numpy, and I could have used one of many well-made packages for training classifiers, but the aim of this project is to write, train, and test this algorithm from scratch. The only module imported is <i>random</i>.
<br><br>
Results:<br>
With weight averaging implemented, the classifier performs around 89% accuracy and takes around 8-14 iterations to train. This is roughly a 1 percent improvement from the non-averaged perceptron before.<br>
The dev set is used here to avoid overfitting the model to the training set. However, accuracy on the dev set seemed to increase with each iteration. Perhaps this would change if we had a larger training set.<br><br>
Accuracy: 89%

In [1]:
import random

# ETL

In [2]:
# Load the Training Data
train_raw = [line.strip() for line in open('./data/hw3data/restaurant_train.txt')]

In [3]:
# Load the Testing Data
test_raw = [line.strip() for line in open('./data/hw3data/restaurant_test.txt')]

In [4]:
# Load the Dev Data
dev_raw = [line.strip() for line in open('./data/hw3data/restaurant_dev.txt')]

### Vectorize the training data, test data, and dev data

In [5]:
# count of features
feature_count = 9492

# count of classes
class_count = 3

In [6]:
def vectorize_raw_data(raw_dataset, feature_count, class_count):
    """
    Parse a list of feature vectors training_data with correct 
    label first from raw data
    Ex. [[0, [0, 1, 1...]], [1, [1, 1, 0...]], [2, [1, 0, 1...]]]
    
    Input, list: lines from raw txt data
    Returns, list: list of feature vectors with their labels first
    Ex. [[0, [0, 1, 1...]], [1, [1, 1, 0...]], [2, [1, 0, 1...]]]"""
    
    # list of labels, feature vectors to be returned
    cleaned_data = []
    
    # Extract each feature vector from the raw training data
    for line in raw_dataset:
        feature_vector_raw = []
        feature_vector_raw = [pair for pair in line.split()]
        # pop first item, this is the class
        class_id = int(feature_vector_raw.pop(0))

        feature_dict = {}
        for fx in feature_vector_raw:
            pair = [int(x) for x in fx.split(':')]
            feature_dict[pair[0]] = pair[1]

        # initialize vector as all zeros
        empty_vec = [0] * feature_count  # total number of features

        # populate the vector with ones for each feature
        for key in feature_dict:
            empty_vec[key] = feature_dict[key]
        feature_vector = [class_id]
        feature_vector.append(empty_vec)

        # add the vector to the training/test/dev set total
        cleaned_data.append(feature_vector)
    return cleaned_data


In [7]:
training_data = vectorize_raw_data(train_raw, feature_count=feature_count, class_count=class_count)

In [8]:
testing_data = vectorize_raw_data(test_raw, feature_count=feature_count, class_count=class_count)

In [9]:
dev_data = vectorize_raw_data(dev_raw, feature_count=feature_count, class_count=class_count)

### Initialize the weight vectors as all zeros

In [10]:
w = [[0] * feature_count] * class_count

### List of labels or classes

In [11]:
classes = list(range(class_count))

# Classify

### Define the mathmatical functions that will be used

Dot Product

In [12]:
def dotproduct(v1, v2):
    """
    Get the dot product of two one-dimensional vectors
    returns: dotproduct, int
    """
    if len(v1) != len(v2):
        return "ERROR: Vectors must be the same length"
    zipped = list(zip(v1, v2))
    return sum([i[0]*i[1] for i in zipped])

Component Addition of Two Vectors

In [13]:
def add_vectors(v1, v2):
    """
    Get the component sum of two one-dimensional vectors
    returns: sum, int
    """
    if len(v1) != len(v2):
        return "ERROR: Vectors must be the same length"
    zipped = list(zip(v1, v2))
    return [i[0]+i[1] for i in zipped]

Component Subtraction of Two Vectors

In [14]:
def subtract_vectors(v1, v2):
    """
    Get the component difference of two one-dimensional vectors
    returns: difference, int
    """
    if len(v1) != len(v2):
        return "ERROR: Vectors must be the same length"
    zipped = list(zip(v1, v2))
    return [i[0]-i[1] for i in zipped]

Divide a Vector by a single integer

In [15]:
def divide_vector(vec, denominator):
    """
    Divide a vector by a single integer or fload
    Returns list: returns a vector
    """
    return [i / denominator for i in vec]

Get the index of the highest value in a list

In [16]:
def get_max_value_index(l):
    max_value = max(l)
    max_index = l.index(max_value)
    return max_index

### Define the classification function

In [17]:
def classify_three_class(fx,w):
    """
    Returns the label with the highest dot product as the predicted label.
    
    Input fx: list. A feature vector 
    Input w: list of weight vectors
    Returns: int. Predicted class label 
    """
    # get dot product of fx and each weight vector
    dp0 = dotproduct(fx, w[0])
    dp1 = dotproduct(fx, w[1])
    dp2 = dotproduct(fx, w[2])
    dotproducts = [dp0, dp1, dp2]
    # check if all dot products are equal
    if dp0 == dp1 == dp2:
        return random.choice([0,1,2])
    else:
        return get_max_value_index(dotproducts)

# Train the averaged perceptron
Checking the dev set after every iteration

In [18]:
def evaluate_accuracy(test_or_dev, w):
    # This function evaluates the accuracy of the model on 
    # either the testset or the devset# either the testset or the devset
    errors = 0
    dataset_size = len(test_or_dev)
    for correct_label, fx in test_or_dev:
        predicted_label = classify_three_class(fx, w)
        if correct_label != predicted_label:
            errors += 1
    return (dataset_size - errors) / dataset_size

Main loop - Train the data. Check against dev set after every iteration to avoid overfitting

In [19]:
errors = 1
iterations = 0
w_sum = w

print('ACCURACY:\n\nTRAIN\t| DEV\n------------------')
while errors > 0:
    errors = 0
    for correct_label, fx in training_data:
        predicted_label = classify_three_class(fx, w)
        if correct_label != predicted_label:
            w[correct_label] = add_vectors(w[correct_label], fx)
            w[predicted_label] = subtract_vectors(w[predicted_label], fx) # subtract features from it
            errors += 1
    # print('ERRORS:', errors)  # this number should go down
    iterations += 1
    # total sum of weight vectors for perceptron averaging
    w_sum = [add_vectors(w_sum[i],w[i]) for i in range(class_count)]
    
    # check against the dev set
    dev_eval = evaluate_accuracy(dev_data, w)
    train_eval = (len(training_data) - errors) / len(training_data)
    print(train_eval, '\t| ', dev_eval)
    
# print accuracy table to watch as accuracy increases with each iteration
# if dev accuracy starts to drop, halt training to avoid overfit
print('\nITERATIONS:', iterations)

# after loop completes, compute the averaged weights
w_avg = [divide_vector(w_sum[i], iterations) for i in range(class_count)]

ACCURACY:

TRAIN	| DEV
------------------
0.681 	|  0.86
0.9 	|  0.908
0.954 	|  0.832
0.973 	|  0.84
0.983 	|  0.836
0.987 	|  0.892
0.988 	|  0.868
1.0 	|  0.868

ITERATIONS: 8


We don't see a significant dip in accuracy on the dev set as we iterate. Accuracy acctually increases. So I will let the perceptron train until completion (at zero training errors in an iteration). If dev accuracy were to drop significantly we would halt the training before it reaches zero errors.

In [20]:
# ACCURACY:

# TRAIN	| DEV
# ------------------
# 0.699 	|  0.78
# 0.894 	|  0.868
# 0.948 	|  0.864
# 0.972 	|  0.88
# 0.978 	|  0.876
# 0.99 	|  0.876
# 0.996 	|  0.892
# 0.993 	|  0.892
# 0.996 	|  0.892
# 0.998 	|  0.892
# 1.0 	|  0.892

# ITERATIONS: 11

# Testing

Perceptron Accuracy: 0.876 - 0.888

In [21]:
evaluate_accuracy(testing_data, w)

0.876

Averaged Perceptron Accuracy 0.892 - 0.894

In [22]:
evaluate_accuracy(testing_data, w_avg)

0.894

# Conclusion

Averaging the weights from each iteration improves accuracy by roughly 1 percent. I expected to see a decrease in accuracy on the dev set before the perceptron reached zero errors, but accuracy on the dev set seemed to increase with each iteration. Perhaps this would change if we had a larger training set.